# Uber Prize Starter Kit Python Utilities: Tutorial

To simplify interaction with the Docker-based simulation execution and evaluation, we've provided a set of Python utilities (located in the `/utilities` folder of the Starter-Kit repository). This notebook demonstrates how they may be used to accomplish the following tasks:

 - Starting a simulation or several simulations
 - Checking simulation completion
 - Retrieving the simulation score in a convenient Pandas `DataFrame` format.
 - Generating fake data 

*Note*: It is assumed that this notebook is started from the `/examples` folder.

In [ ]:

# Adding the module to the path for future import
import sys
import os
import docker
from pathlib import Path
# Note that the following is idempotent when this notebook is run from "/examples"
os.chdir('../utilities')
%load_ext autoreload
%autoreload 2

## The `competition_executor` Module

A `CompetitionContainerExecutor` object may be used to start, stop, and gather information about containers running simulations and/or completed simulation scores and stats.

In [ ]:
from competition_executor import CompetitionContainerExecutor

path_input = (Path(Path.cwd()).parent / "submission-inputs").absolute()
path_ouput = (Path(Path.cwd()).parent / "output").absolute()

my_executor = CompetitionContainerExecutor(input_root=path_input,
                                  output_root=path_ouput)

# Note: Instantiating a CompetitionContainerExecutor with path arguments is not strictly necessary. Each simulation
# can be run with its own set of input and/or output arguments. However, if you prefer to designate
# a single directory for inputs and/or a single directory for outputs, then, for convenience, 
# you may pass those arguments in here and avoid re-entering them for each simulation run.

Run the simulation using the `run_sumulation` method. For example:

In [ ]:
# Note: the `submission_id`s for each run must be unique; however, this cell may be run idempotently
try:
    my_submission = my_executor.client.containers.get('my_submission')
    my_submission.stop()
    my_submission.remove()
    print("Found container from previous run, removing and creating new sim container...")
except docker.errors.NotFound:
    print("Creating new simulation container...")
    
my_executor.run_simulation('my_submission', num_iterations=2, num_cpus=4)

# Note that a dictionary indicating which containers IDs already exist can be accessed.
for key, value in my_executor.containers.items():
    print("Container ID : {0}".format(key))

You may view the **logs** of a specific simulation as follows:

In [ ]:
logs = my_executor.output_simulation_logs('my_submission')

Submissions run for a certain amount of time. If you interrupt it before the end, you will not get any outputs. You can **check if a submission is finished** with the following method:

In [ ]:
my_executor.check_if_submission_complete('my_submission')

When a simulation run is done, you can import its **results**:

In [ ]:
scores, stats = my_executor.get_submission_scores_and_stats('my_submission')

The **scores** and **statistics** are stored in pandas DataFrames which contains the information described [here](https://github.com/vgolfier/Uber-Prize-Starter-Kit/blob/master/docs/Understanding_the_outputs_and_the%20scoring_function.md).

At the end of any run and to avoid any conflicts between submission ids (particularly if you want to reuse names), it is advised to **clean up containers** calling the following method:

In [ ]:
my_executor.stop_all_simulations()

## The `input_sampler` module:

Randomly sampled data may be used to either initialize the re-planning algorithm or otherwise test the simulation. The `input_sampler` has been provided towards this end. This subsection provides an example of how synthetic random input files may be generated for each of the available input policies to the simulation.

In [10]:
from input_sampler import *
# Specify the common string for the scenario name 
# (currently only "siouxfalls", which refers to the Sioux Falls scenario).
SCENARIO_NAME = 'sioux_faux'
agency = 'sioux_faux_bus_lines'

# Set the paths appropriately
DIR = Path(__name__).absolute().parent.parent
sys.path.append(str(DIR))
DATA_DIR = DIR / 'reference-data/'

agency_dict = scenario_agencies(DATA_DIR,SCENARIO_NAME)
    # Create a lazy cache of GTFS data for the agency:
sf_gtfs_manager = AgencyGtfsDataManager(agency_dict[agency])

# Sample each input five (5) times (naming of functions is indicative of type of input sampled).
freq_df = sample_frequency_adjustment_input(5, sf_gtfs_manager)
mode_subsidy_df = sample_mode_subsidies_input(5)
vehicle_fleet_mix_df = sample_vehicle_fleet_mix_input(5, sf_gtfs_manager)

# Generated inputs may now be saved in, say, /submission-inputs or some other directory. 
# Remember the location of this directory when executing new simulations.

In [11]:
vehicle_fleet_mix_df

,agencyId,routeId,vehicleType
0,217,1348,CAR-TYPE-DEFAULT
1,217,1340,BUS-STD-HD
2,217,1343,BUS-STD-HD
3,217,1341,BUS-SMALL-HD
4,217,1351,CAR-TYPE-DEFAULT


In [18]:
min_headway_minutes_by_route=

In [16]:
sf_gtfs_manager.trips['min_headway_minutes'] = sf_gtfs_manager.qumin_headway_minutes_by_route 

,route_id,service_id,trip_short_name,trip_headsign,direction_id,block_id,shape_id,bikes_allowed,wheelchair_accessible,trip_type,drt_max_travel_time,drt_avg_travel_time,drt_advance_book_min,drt_pickup_message,drt_drop_off_message,continuous_pickup_message,continuous_drop_off_message
trip_id,,,,,,,,,,,,,,,,,
t_75337_b_219_tn_5,1340,c_676_b_219_d_31,NaN,(loop),NaN,NaN,p_3333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t_75337_b_219_tn_4,1340,c_676_b_219_d_31,NaN,(loop),NaN,NaN,p_3333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t_75337_b_219_tn_3,1340,c_676_b_219_d_31,NaN,(loop),NaN,NaN,p_3333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t_500920_b_219_tn_0,1340,c_676_b_219_d_31,NaN,(loop),NaN,NaN,p_3333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t_500921_b_219_tn_0,1340,c_676_b_219_d_31,NaN,(loop),NaN,NaN,p_3333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t_500922_b_219_tn_0,1340,c_676_b_219_d_31,NaN,(loop),NaN,NaN,p_3333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t_500923_b_219_tn_0,1340,c_676_b_219_d_31,NaN,(loop),NaN,NaN,p_3333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t_75335_b_219_tn_1,1340,c_676_b_219_d_31,NaN,(loop),NaN,NaN,p_3333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t_75335_b_219_tn_2,1340,c_676_b_219_d_31,NaN,(loop),NaN,NaN,p_3333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
min_headway_mins=sf_gtfs_manager.trips.join(sf_gtfs_manager.routes.min_headway_minutes,on='route_id').min_headway_minutes

In [29]:
trip_ids = pd.Series(sf_gtfs_manager.trips.sample(5).index.values)

In [35]:
[np.random.choice(range(m*60,7200,60)) for m in  min_headway_mins[trip_ids].values]

[3300, 1920, 6720, 4440, 5400]